Sumário:

1. <a href="#sec1"> Importações </a>
2. <a href="#sec2"> Carregando dados </a>
3. <a href="#sec3"> Visualização e exploração dos dados </a>
    * <a href="#sec30"> Remodelando formato dos dados </a>
    * <a href="#sec31"> Visualização </a>
    * <a href="#sec32"> Imagem média por classe </a>
    * <a href="#sec33"> Pixel médio por classe </a>
4. <a href="#sec4"> Pré-processamento </a>
    * <a href="#sec41"> Rótulos </a>
    * <a href="#sec42"> Normalização </a>
    * <a href="#sec43"> Divisão </a>
5. <a href="#sec5"> Definindo e treinando o modelo </a>
6. <a href="#sec6"> Avaliação do modelo </a>
7. <a href="#sec7"> Previsão </a>
    * <a href="#sec71"> Gerando a submissão </a>
8. <a href="#sec8"> Possíveis passos futuros </a>


<a id="sec1"></a>
## 1. Importando bibliotecas

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

<a id="sec2"></a>
## 2. Importando arquivos

In [2]:
df_train.head()

NameError: name 'df_train' is not defined

In [ ]:
df_test.head()

In [ ]:
x_train_temp = np.array(df_train.iloc[:, 1:])
y_train = np.array(df_train.iloc[:, 0])

x_test = np.array(df_test)

<a id="sec3"></a>
## 3. Vizualização e exploração dos dados

<a id="sec30"></a>
### Remodelando formato dos dados

In [ ]:
N_train_samples = x_train_temp.shape[0]
x_train = x_train_temp.reshape(N_train_samples, 28, 28, 1)

<a id="sec31"></a>
### Visualização 

In [ ]:
plt.figure(figsize=(30, 15))

n = 10

for i in range(n):
    plt.subplot(1, n, i+1)
    img = x_train[i]
    plt.imshow(img, cmap='Greys')
plt.show()

<a id="sec32"></a>
### Imagem média por classe

In [ ]:
def get_class_arrays(class_index):
    return x_train[y_train==class_index],y_train[y_train==class_index]

In [ ]:
def get_mean_images():
    for i in range(10):
        class_arrayX, class_arrayY = get_class_arrays(i)
        mean_image = class_arrayX.mean(axis=0)
        plt.subplot(2, 5, i+1)
        plt.axis("off")
        plt.title(i)
        plt.imshow(mean_image.squeeze())
        
plt.figure(figsize=(15, 5))
get_mean_images()

<a id="sec33"></a>
### Pixel médio por classe

Média de todos o pixels, de todas as imagens, por classe. Note que o valor dos pixels estão em uma faixa de valores de 0-255.

In [ ]:
def get_mean_barchart():
    mean_values =[]
    for i in range(10):
        class_arrayX,class_arrayY = get_class_arrays(i)
        mean_values.append(class_arrayX.mean())
    
    plt.bar(np.arange(10), mean_values)
    plt.xticks(np.arange(10))
    
plt.figure(figsize=(15, 5))
get_mean_barchart()

<a id="sec4"></a>
## 4. Pré-processamento

<a id="sec41"></a>
### Classe

In [ ]:
y_train

In [ ]:
y_cat_train = to_categorical(y_train)
y_cat_train

<a id="sec42"></a>
### Normalização dos valores de cada pixel

In [ ]:
x_train = x_train/255

##### <a id="sec43"></a>
### Separação dos dados

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_cat_train, test_size=0.2, random_state=42)

<a id="sec5"></a>
## 5. Definindo e treinando o modelo

In [ ]:
model = Sequential()

model.add(Conv2D(6,(5,5), activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(16, (5,5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2))),
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=False)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',
                           patience=5)

In [ ]:
# TRAIN MODEL

model.fit(X_train,
          Y_train, 
          epochs=50, 
          validation_data=(X_val, Y_val),
          callbacks=[early_stop])

<a id="sec6"></a>
## 6. Avaliando o modelo

In [ ]:
losses = pd.DataFrame(model.history.history)
losses

In [ ]:
losses[['accuracy','val_accuracy']].plot()

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
pred_val = np.argmax(model.predict(X_val), axis=1)
real_val = np.argmax(Y_val, axis=1)

In [ ]:
print(classification_report(real_val, pred_val))

In [ ]:
confusion_matrix(real_val, pred_val)

<a id="sec7"></a>
## 7. Predição

In [ ]:
N_test_samples = x_test.shape[0]

x_test_new = x_test.reshape(N_test_samples, 28, 28, 1)
x_test_new = x_test_new/255

x_test_new.shape

In [ ]:
predictions = model.predict(x_test_new)

print(predictions.shape)

predictions

In [ ]:
final_pred = np.argmax(predictions, axis=1)

final_pred

In [ ]:
# Visualize some test prediction and corresponding digit 

test_index = 10

plt.imshow(x_test_new[test_index], cmap='Greys')
print(f"Predicted value: {final_pred[test_index]}\n")


<a id="sec71"></a>
### Gerando a submissão

In [ ]:
pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [ ]:
submission_dict = {"ImageId": [i+1 for i in range(28000)],
                   "Label": final_pred}

sub_df = pd.DataFrame(submission_dict)

sub_df

In [ ]:
sub_df.to_csv("submission.csv", index=False)

<a id="sec8"></a>
## 8. Possíveis passos futuros

* Desenvolver uma análise exploratória mais profunda
* Realizar testes para encontrar uma seleção de variáveis mais relevante
* Expandir o conjunto de dados utilizado
* Implementar validação cruzada
* Implementar uma CNN com arquitetura diferente
* Implementar grid search
* Comparar o aproveitamento com o de metodologias menos elaboradas